In [6]:
import tensorflow as tf

import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tqdm import tqdm
import pandas as pd

from IPython import display

In [14]:
class GAN:
    def __init__(self, input_shape=(28, 28, 1), latent_dim=100):
        self.input_shape = input_shape
        self.latent_dim = latent_dim
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()
        self.gan = self.build_gan()

    def build_generator(self):
        model = tf.keras.Sequential()
        model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(self.latent_dim,)))
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU())

        model.add(layers.Reshape((7, 7, 256)))
        assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

        model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
        assert model.output_shape == (None, 7, 7, 128)
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU())

        model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
        assert model.output_shape == (None, 14, 14, 64)
        model.add(layers.BatchNormalization())
        model.add(layers.LeakyReLU())

        model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
        assert model.output_shape == (None, self.input_shape[0], self.input_shape[1], self.input_shape[2])
        
        return model

    def build_discriminator(self):
    
        model = tf.keras.Sequential()
        model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                        input_shape=self.input_shape))
        model.add(layers.LeakyReLU())
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
        model.add(layers.LeakyReLU())
        model.add(layers.Dropout(0.3))

        model.add(layers.Flatten())
        model.add(layers.Dense(1))

        return model

    def build_gan(self):
        self.discriminator.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
        self.discriminator.trainable = False
        gan = tf.keras.Sequential()
        gan.add(self.generator)
        gan.add(self.discriminator)
        gan.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
        return gan
    
    def generate_and_save_images(self, model, epoch, test_input):
        predictions = model(test_input, training=False)
        fig = plt.figure(figsize=(4,4))
        for i in range(predictions.shape[0]):
            plt.subplot(4, 4, i+1)
            plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
            plt.axis('off')
        plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
        plt.show()

    @tf.function
    def train_step(self, images):
        noise_dim = 100
        batch_size = images.shape[0]

        # Train the discriminator
        noise = tf.random.normal([batch_size, noise_dim])
        generated_images = self.generator(noise, training=True)

        real_output = self.discriminator(images, training=True)
        fake_output = self.discriminator(generated_images, training=True)

        d_loss = self.discriminator_loss(real_output, fake_output)
        g_loss = self.generator_loss(fake_output)

        d_gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
        g_gradients = tape.gradient(g_loss, self.generator.trainable_variables)

        self.d_optimizer.apply_gradients(zip(d_gradients, self.discriminator.trainable_variables))
        self.g_optimizer.apply_gradients(zip(g_gradients, self.generator.trainable_variables))

        return {'d_loss': d_loss, 'g_loss': g_loss}

    def train(self, data, epochs=50, batch_size=128):
        for epoch in range(epochs):
            for image_batch in data:
                self.train_step(image_batch)
            if (epoch + 1) % 15 == 0:
                self.generate_and_save_images(self.generator, epoch + 1, self.seed)
            print('Epoch: ', epoch)
        self.generate_and_save_images(self.generator, epochs, self.seed)

In [7]:
df = pd.read_csv(os.environ.get("DATASET_PATH"), header=None)
df.head()
train_label = df[0]
train_images = df.drop(0, axis=1)
augment_images = []
for i in train_images.index:
    pixels = train_images.loc[i].values
    image = np.array(pixels).reshape(28,28)
    rotated_image = np.rot90(image, k=-1)
    flipped_horizontal = np.fliplr(rotated_image)
    augment_images.append(flipped_horizontal)

train_images = np.array(augment_images)
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]

In [8]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [16]:
dcgan = GAN()
dcgan.train(train_dataset, epochs=100, batch_size=256)